Dilated CNN (Temporal Convolutional Network - TCN)
Uses dilated convolutions to increase receptive field without increasing the number of parameters.

Better for capturing long-term dependencies compared to regular CNNs.

TCNs are strong alternatives to RNNs/LSTMs for sequence modeling.

TCN for Horizon 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    fft_values = fft(df[target_col].dropna().values)
    fft_real = np.real(fft_values)[:len(df[target_col])]
    fft_imag = np.imag(fft_values)[:len(df[target_col])]
    df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 1.0       | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 58.87     | 0.237     | 99.91     | 32.34     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 1.0       | 17.24     | 0.4906    | 50.56     | 33.63     |
| 9         | 1.0       | 16.41     | 0.4657    | 98.87     | 127.3     |
| 10        | 1.0       | 18.2      | 0.4567    | 99.88     | 32.67     |
| 11        | 1.0       | 63.94     | 0.4128    | 50.58     | 125.6     |
| 12  


===== Evaluation =====
Accuracy: 0.9656
Report:
               precision    recall  f1-score   support

           0     1.0000    0.6071    0.7556        56
           1     0.9151    0.9875    0.9499       240
           2     0.9938    0.9938    0.9938       487
           3     0.9091    1.0000    0.9524        30

    accuracy                         0.9656       813
   macro avg     0.9545    0.8971    0.9129       813
weighted avg     0.9679    0.9656    0.9629       813

Confusion Matrix:
 [[ 34  22   0   0]
 [  0 237   3   0]
 [  0   0 484   3]
 [  0   0   0  30]]

===== Model Metrics =====
Training Time: 49.70s
Inference/sample: 0.000911s
Testing Time: 0.74s
Model Size: 0.42 MB
Trainable Params: 102,752
RAM Usage: 927.19 MB


TCN for horizon 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=1, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()

    # Ensure kernel_size is smaller than input size
    if input_shape[1] < kernel_size:
        raise ValueError(f"Kernel size {kernel_size} cannot be larger than the input length {input_shape[1]}.")

    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9919    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9959    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 17.03     | 0.1144    | 99.59     | 126.0     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 1.0       | 23.86     | 0.2556    | 51.96     | 114.0     |
| 9         | 1.0       | 16.02     | 0.4024    | 98.89     | 93.7      |
| 10        | 1.0       | 63.97     | 0.297     | 99.67     | 95.01     |
| 11        | 1.0       | 39.67     | 0.3506    | 99.21     | 109.6     |
| 12  


===== Evaluation =====
Accuracy: 0.9643
Report:
               precision    recall  f1-score   support

           0     0.9091    0.4762    0.6250        42
           1     0.9087    0.9690    0.9379       226
           2     0.9903    1.0000    0.9951       509
           3     1.0000    1.0000    1.0000        36

    accuracy                         0.9643       813
   macro avg     0.9520    0.8613    0.8895       813
weighted avg     0.9638    0.9643    0.9603       813

Confusion Matrix:
 [[ 20  22   0   0]
 [  2 219   5   0]
 [  0   0 509   0]
 [  0   0   0  36]]

===== Model Metrics =====
Training Time: 24.29s
Inference/sample: 0.000397s
Testing Time: 0.32s
Model Size: 0.16 MB
Trainable Params: 35,824
RAM Usage: 930.87 MB


TCN for horizon 3

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h3'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9919    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9919    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9919    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9959    | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 0.9959    | 16.54     | 0.4948    | 74.52     | 107.4     |
| 8         | 0.9959    | 30.76     | 0.2844    | 99.76     | 109.5     |
| 9         | 0.9959    | 62.74     | 0.4618    | 98.04     | 101.0     |
| 10        | 0.9959    | 62.47     | 0.3844    | 99.36     | 127.1     |
| 11        | 0.9959    | 16.15     | 0.1681    | 97.47     | 86.36     |
| 12  


===== Evaluation =====
Accuracy: 0.9656
Report:
               precision    recall  f1-score   support

           0     1.0000    0.3704    0.5405        27
           1     0.9241    0.9718    0.9474       213
           2     0.9886    0.9905    0.9895       524
           3     0.9074    1.0000    0.9515        49

    accuracy                         0.9656       813
   macro avg     0.9550    0.8332    0.8572       813
weighted avg     0.9672    0.9656    0.9613       813

Confusion Matrix:
 [[ 10  17   0   0]
 [  0 207   6   0]
 [  0   0 519   5]
 [  0   0   0  49]]

===== Model Metrics =====
Training Time: 33.93s
Inference/sample: 0.000641s
Testing Time: 0.52s
Model Size: 0.26 MB
Trainable Params: 59,676
RAM Usage: 1273.87 MB


TCN for horizon 4

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h4'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9919    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9959    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9878    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9878    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9919    | 42.17     | 0.3911    | 81.09     | 105.0     |
| 7         | 0.9959    | 35.13     | 0.2734    | 86.66     | 90.18     |
| 8         | 0.9919    | 63.97     | 0.4477    | 55.79     | 127.9     |
| 9         | 0.9919    | 38.77     | 0.4718    | 99.57     | 80.95     |
| 10        | 0.9959    | 24.43     | 0.3747    | 93.81     | 98.93     |
| 11        | 0.9959    | 17.78     | 0.4417    | 79.88     | 96.61     |
| 12  


===== Evaluation =====
Accuracy: 0.9545
Report:
               precision    recall  f1-score   support

           0     1.0000    0.4615    0.6316        13
           1     0.9722    0.8794    0.9235       199
           2     0.9623    0.9908    0.9763       541
           3     0.8429    0.9833    0.9077        60

    accuracy                         0.9545       813
   macro avg     0.9443    0.8288    0.8598       813
weighted avg     0.9565    0.9545    0.9528       813

Confusion Matrix:
 [[  6   5   2   0]
 [  0 175  18   6]
 [  0   0 536   5]
 [  0   0   1  59]]

===== Model Metrics =====
Training Time: 26.32s
Inference/sample: 0.000403s
Testing Time: 0.33s
Model Size: 0.26 MB
Trainable Params: 59,676
RAM Usage: 1637.45 MB
